## Given a connected graph, is there a path connecting p and q?

![Dynamic Connectivity](imgs/dynamic_connectivity.PNG)

## Quick Find

The **quick find** (or **eager approach**) implements data structure like this: 
- Integer array id[] of length N
- Interpretation: p and q are connected iff they have the same id

and two functions:
- connected. Check if p and q are connected
- union. To merge components containing p and q, change all entries whose id equals id[p] to id[q] (basically connect two nodes).

In [6]:
class QuickFindUF:
    def __init__(self, N):
        self.id = [i for i in range(N)] #initialize ids
    
    def connected(self, p, q):
        return self.id[p] == self.id[q]
    
    def union(self, p, q):
        p_id = self.id[p]
        q_id = self.id[q]
        # assign all entries whose id equals id[p] to id[q]
        for i in range(len(self.id)):
            if self.id[i] == p_id:
                self.id[i] = q_id

Example: (0, 1) (2, 3, 4) (5)

In [8]:
quick_find = QuickFindUF(6)
quick_find.union(0, 1)
quick_find.union(2, 3)
quick_find.union(3, 4)

In [19]:
print(quick_find.id)

[1, 1, 4, 4, 4, 5]


In [9]:
print(quick_find.connected(0, 1))

True


In [10]:
print(quick_find.connected(2, 4))

True


In [12]:
print(quick_find.connected(3, 5))

False


*Note that this quick find algorithm takes O(1) time for checking connectivity, 
but take O(N^2) time for initializing the unions. Therefore it is quadratic and not scalable

## Quick Union (Lazy Approach)

The **quick find** (or **eager approach**) implements data structure like this: 
- Integer array id[] of length N
- Interpretation: id[i] is the parent of i

and two functions:
- find. Check if p and q have the same root (connected).
- union. To merge components containing p and q, set the id of p's root to the id of q's root.

In [15]:
class QuickUnionUF():
    def __init__(self, N):
        self.id = [i for i in range(N)] #initialize ids
    
    def root(self, i):
        # keep finding parent util reaching root
        while(i != self.id[i]):
            i = self.id[i] 
        return i
    
    def find(self, p, q):
        return self.root(p) == self.root(q)
    
    def union(self, p, q):
        i = self.root(p)
        j = self.root(q)
        self.id[i] = j

In [16]:
quick_union = QuickUnionUF(6)
quick_union.union(0, 1)
quick_union.union(2, 3)
quick_union.union(3, 4)

In [20]:
print(quick_union.id)

[1, 1, 3, 4, 4, 5]


In [21]:
print(quick_find.connected(0, 1))

True


In [22]:
print(quick_find.connected(2, 4))

True


In [23]:
print(quick_find.connected(3, 5))

False


*Note that this lazy approach has worst case of O(N) time for finding a root (a deep tree where each level has only one node). Algo can be slow due to the imbalance of trees.

**Improvement #1**: Weighted trees
- Always attach smaller tree to bigger tree to avoid building very deep tree

In [28]:
class WeightedQU():
    def __init__(self, N):
        self.id = [i for i in range(N)] #initialize ids
        self.sz = [1 for i in range(N)] #keep track of size of trees
        
    def root(self, i):
        # keep finding parent util reaching root
        while(i != self.id[i]):
            i = self.id[i] 
        return i
    
    def find(self, p, q):
        return self.root(p) == self.root(q)
    
    def union(self, p, q):
        i = self.root(p)
        j = self.root(q)
        if self.sz[i] < self.sz[j]:
            self.id[i] = j
            self.sz[j] += self.sz[i]
        else:
            self.id[j] = i
            self.sz[i] += self.sz[j]

In [29]:
weighted_qu = WeightedQU(6)
weighted_qu.union(0, 1)
weighted_qu.union(2, 3)
weighted_qu.union(3, 4)

In [30]:
print(weighted_qu.id)
print(weighted_qu.sz)

[0, 0, 2, 2, 2, 5]
[2, 1, 3, 1, 1, 1]


In [31]:
print(quick_find.connected(0, 1))
print(quick_find.connected(2, 4))
print(quick_find.connected(3, 5))

True
True
False


**Improvement #2**: Path compression
- Two-pass implementation: add second loop to root() to set the id[] of each examined node to the root.
- Simpler one-pass variant: Make every other node in path point to its grandparent (thereby halving path length).

Here we implement the second option which is simpler (one line of code added)

In [32]:
class WeightedQUPC():
    def __init__(self, N):
        self.id = [i for i in range(N)] #initialize ids
        self.sz = [1 for i in range(N)] #keep track of size of trees
        
    def root(self, i):
        # keep finding parent util reaching root
        while(i != self.id[i]):
            self.id[i] = self.id[self.id[i]] #Make every other node in path point to its grandparent
            i = self.id[i] 
        return i
    
    def find(self, p, q):
        return self.root(p) == self.root(q)
    
    def union(self, p, q):
        i = self.root(p)
        j = self.root(q)
        if self.sz[i] < self.sz[j]:
            self.id[i] = j
            self.sz[j] += self.sz[i]
        else:
            self.id[j] = i
            self.sz[i] += self.sz[j]

In [33]:
weighted_qupc = WeightedQUPC(6)
weighted_qupc.union(0, 1)
weighted_qupc.union(2, 3)
weighted_qupc.union(3, 4)

In [34]:
print(quick_find.connected(0, 1))
print(quick_find.connected(2, 4))
print(quick_find.connected(3, 5))

True
True
False
